In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components



Autosaving every 180 seconds


In [2]:
# Default functions
        
def load_csv(fname):
    with open(fname, newline='') as csvfile:
        data = list(csv.reader(csvfile))

    labels = data[1]
    #print ("data labels: ", labels)
    #print ("column vals: ", data[2])

    # load values
    data_array = np.array(data[3:])
    #print ("# of datapoints (x,y,likelihood): ", data_array.shape)

    # 
    #labels = ['fnose','f_leye','f_reye','f_lear','f_rear','f_',
    #         'male_nose','male_left_ear','male_right_ear','male_base_of_tail',
    #          'pup_shaved_nose','pup_shaved_left_ear','pup_shaved_right_ear','pup_shaved_base_of_tail',
    #          'pup_noshave_nose','pup_noshave_left_ear','pup_noshave_right_ear','pup_noshave_base_of_tail'             
    #         ]
    
    labels = labels[1:]
    
    traces = []
    traces_nan = []
    # zero out low quality DLC values
    for idx in range(1,len(labels)-1,3):
        #print ("idx: ", idx)
        #print (data_array[1:,idx:idx+3])
        #print (data_array[1:,idx:idx+3].shape)

        temp = data_array[1:,idx:idx+3]
        idx1 = np.where(temp=='')[0]
        temp[idx1]=0
        temp = temp.astype(np.float)# np.array(temp)
        #print (idx, "TEMP: ", temp.shape)
        #temp.replace("''",'0')
        

        # replace low likelihoods with median
        likelihoods = temp[:,2]
        idx2 = np.where(likelihoods<0.8)[0]
        temp[idx2,0]=np.median(temp[:,0])
        temp[idx2,1]=np.median(temp[:,1])
        traces.append(temp.copy())
        
        temp[idx2,0]=np.nan
        temp[idx2,1]=np.nan
        traces_nan.append(temp.copy())

    return traces, labels, traces_nan

# function that does search forward steps: 
def search_forward(data_assembled_fixed,
                  traces_inf,
                  threshold_p,
                  dist_threshold,
                  data_assembled_all_features,
                  selected_feature,
                  comments=False):

    # count # of merges
    n_merges = 0
    
    # load traces
    tracex = data_assembled_fixed[:,0]
    tracey = data_assembled_fixed[:,1]

    # find assembled locations with high prob. 
    probs = data_assembled_fixed[:,2]
    idx = np.where(probs>threshold_p)[0]
    
    # if no values over threshold skip chunk;
    if idx.shape[0]==0:
        return data_assembled_fixed, data_assembled_all_features
    
    
    if comments:
        print ("data assembled fixed: ", data_assembled_fixed.shape)
        print ("traces inf: ", traces_inf.shape)
        print ("probs: ", probs[:10])
        print ("IDX: ", idx.shape)

    # find ends of continous labeled segments
    # old method
    #diffs = idx[1:]-idx[:-1]
    #seg_ends = np.where(diffs>1)[0]
    seg_ends=[]
    for k in range(idx.shape[0]-1):
        if (idx[k+1]-idx[k])>1:
            seg_ends.append(k)
    # append the last value if it's not at the end of the data
    if idx.shape[0]<data_assembled_fixed.shape[0]:
        if idx.shape[0]>0:
            seg_ends.append(idx.shape[0]-1)
    #print ("idx shape:" , idx.shape[0], " tracex: ", tracesx.shape[0])
    #print ("Actual last idx values: ", idx[-1])

    seg_ends = np.array(seg_ends)
    #print (seg_ends.shape, idx[-1].shape, idx[-1])
    # add last segment also
    #if idx[-1]!=(tracex.shape[0]-1):
    #    seg_ends.append()
#     if idx.shape[0]>0:
#         seg_ends=np.concatenate((seg_ends, [idx[-1]]),axis=0)
    
    if comments:
        #print ("diffs[ends]: ", diffs[seg_ends])
        if seg_ends.shape[0]>0:
            print (seg_ends)
            print ("seg ends: ", idx[seg_ends])

    # propagate forward
    # loop over all ends of continous segments and search forward for min distances
    for l in range(seg_ends.shape[0]):
        idx_start = idx[seg_ends[l]]

        # initalized with the first starting location;
        loc0 = np.array([tracex[idx_start], tracey[idx_start]])

        # search forward
        idx_next = idx_start+1
        if idx_next>=1000:
            continue
        while True:
            if comments:
                print ("idx_current: ", idx_next-1 , "/", seg_ends.shape[0],
                       "loc0: ", loc0, " prob: ", probs[idx_next-1])
                print ("idx_next: ", idx_next , 
                       "loc0: ", np.array([tracex[idx_next], tracey[idx_next]]), " prob: ", probs[idx_next])

            # if there is no inference, delete data
            if len(traces_inf[idx_next])==0:
                break
                
            loc_candidates = np.vstack(traces_inf[idx_next]).T
            if comments:
                print ("loc_candidates: ", loc_candidates)
  
            # compute distance between previous assembled (true) location and canadidate
            dist = (loc_candidates.T - loc0)**2
            dist = np.sum(dist, axis=1)
            dist = np.sqrt(dist)

            # minimum distance is less than threshold, add to assembled animal data;
            min_dist = np.min(dist)
            arg_min = np.argmin(dist)
            if comments:
                print ("Dist: ", min_dist, " argmin: ", arg_min, " values; ", loc_candidates.T[arg_min])

            if min_dist < dist_threshold:
                
                # first check if the value has alerady been assigned to an assembled animal;
                if comments:
                    print (" CROSS VALUE SEARCH *****************************************************")
                    print (data_assembled_all_features.shape)
                    print ("data assembled all features; ", data_assembled_all_features[:,idx_next,0])
                    print ("best inference candidate: ", loc_candidates[0][arg_min])
                    print (np.min(np.abs(data_assembled_all_features[:,idx_next, 0]-loc_candidates[0][arg_min])))

                # Chekc if x values are identical between a previously assembled feature and the best inference match
                # if so, check if the probability of the assembled feature is above thrshold (ie. >0 as it's already been set to 0)
                # and skip it; 
                # otherwise, do not inherit the label and exit the segment
                if np.min(np.abs(data_assembled_all_features[:,idx_next,0]-loc_candidates[0][arg_min]))<1E-5:
                    if np.min(np.abs(data_assembled_all_features[:,idx_next,1]-loc_candidates[1][arg_min]))<1E-5:
                        argmin_temp = np.argmin(np.abs(data_assembled_all_features[:,idx_next,0]-
                                                    loc_candidates[0][arg_min]))
                        if data_assembled_all_features[argmin_temp,idx_next,2]>0.0:
                            if comments:
                                print ("Infered value belongs to already assbmeld feature", argmin_temp,
                                      "  with prob: ", data_assembled_all_features[argmin_temp,idx_next,2])
                                print ("data_assembled_all_features[idx_next]: ", data_assembled_all_features.shape)
                                print (np.argmin(np.abs(data_assembled_all_features[:,0]-loc_candidates[0][arg_min])))

                            break

                if comments:
                    print ("replace assembled val at time step: ", idx_next, " at loc: ", loc0 )
                    
                loc0 = np.array([loc_candidates[0][arg_min],
                                loc_candidates[1][arg_min]])
                if comments:

                    print ("      with new location from inference", loc0)

                    print ('data_assembled_fixed[feature][pre]: ', 
                           data_assembled_fixed[idx_next])

                
                # fix the data in progress
                data_assembled_fixed[idx_next]=np.array([loc0[0],  # set x
                                                         loc0[1],  # set y
                                                         1.0])     # set probability
                # fix master list as well
                data_assembled_all_features[selected_feature,idx_next]=np.array([loc0[0],  # set x
                                                         loc0[1],  # set y
                                                         1.0])     # set probability
                
                # metadata printing
                if comments:
                    print ('data_assembled_fixed[feature][post]: ', 
                       data_assembled_fixed[idx_next])

                    print ('')
                    
                    
                idx_next+=1
                n_merges+=1
                # exit if at end of data
                if idx_next>=1000:
                    break

                # exit if reached a chunk that is labled above accepted probability:
                if data_assembled_fixed[idx_next][2]>threshold_p:
                    if comments:
                        print ("***** Point has prob > threshold (moving to next discontious segment) *****")
                        print ("")
                        print ("")
                        print ("")
                    break


            else: #move to the next continous segment if distance to nearest time point is too far
                if comments:
                    print ("***** JUMPING OT NEXT SEG *****")
                    print ("")
                    print ("")
                    print ("")
                break

            #return
                
    print ('********** # OF MERGES: ', n_merges)

    # return fixed data
    return data_assembled_fixed, data_assembled_all_features

In [3]:
fnames = [
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_54_23_358257_compressed/2020-3-16_01_54_23_358257_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02_54_39_170978_compressed/2020-3-16_02_54_39_170978_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03_57_56_902379_compressed/2020-3-16_03_57_56_902379_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_03 54 54 231226_compressed/2020-3-16_03 54 54 231226_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_55_09_841582_compressed/2020-3-16_04_55_09_841582_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_04_58_11_998956_compressed/2020-3-16_04_58_11_998956_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 55 25 305681_compressed/2020-3-16_05 55 25 305681_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_05 58 27 193818_compressed/2020-3-16_05 58 27 193818_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 55 40 714236_compressed/2020-3-16_06 55 40 714236_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_06 58 43 678014_compressed/2020-3-16_06 58 43 678014_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_55_55_775234_compressed/2020-3-16_07_55_55_775234_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_07_59_00_362242_compressed/2020-3-16_07_59_00_362242_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_56_11_096689_compressed/2020-3-16_08_56_11_096689_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_08_59_17_534732_compressed/2020-3-16_08_59_17_534732_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09_59_34_731308_compressed/2020-3-16_09_59_34_731308_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_09 56 26 362091_compressed/2020-3-16_09 56 26 362091_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10_59_50_448686_compressed/2020-3-16_10_59_50_448686_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_10 56 41 406701_compressed/2020-3-16_10 56 41 406701_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_11_56_56_704655_compressed/2020-3-16_11_56_56_704655_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_12 57 12 418305_compressed/2020-3-16_12 57 12 418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_02 57 41 995158_compressed/2020-3-16_02 57 41 995158_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle',
'/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/madeline/march_2/march_16/2020-3-16_01_57_27_327194_compressed/2020-3-16_01_57_27_327194_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'
         ]

In [3]:
fnames = ['/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full.pickle']

In [4]:
# LOAD INFERENCE DATA from full_pickle files and convert to  .npy file
end = None
#end = 1000
for full_pickle in fnames:
    print ("processing :", full_pickle)
    # convert full pickel file to simpler data structure
    fname_out = full_pickle[:-7]+"_traces_inferences.npy"
    if os.path.exists(fname_out)==False:
        from deeplabcut.pose_estimation_tensorflow.lib.inferenceutils import (
                convertdetectiondict2listoflist)
        import pickle, re
        # load pickle and 
        with open(full_pickle, "rb") as file:
            data = pickle.load(file)
        header = data.pop("metadata")
        all_jointnames = header["all_joints_names"]

        #if displayedbodyparts == "all":
        if True:
            numjoints = len(all_jointnames)
            bpts = range(numjoints)

        frame_names = list(data)
        frames = [int(re.findall(r"\d+", name)[0]) for name in frame_names]

        # Convert inference locations to an easier array to parse

        traces_inferences = []
        ctr=0
        start = 0
        if end is None:
            end = len(frame_names)

        for n in trange(start, end, 1):
            #ind = n
            #print (n, frame_names[n])
            #print (data)
            # load inference locations
            traces_inferences.append([])
            dets = convertdetectiondict2listoflist(data[frame_names[n]], bpts)
            for i, det in enumerate(dets):
                traces_inferences[ctr].append([])
                for x, y, p, _ in det:
                    traces_inferences[ctr][i].append([x,y])

            ctr+=1
        traces_inferences = np.array(traces_inferences)
        print (traces_inferences.shape)

        np.save(fname_out, traces_inferences)

    else:
        traces_inferences = np.load(fname_out, allow_pickle=True)
    print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)


processing : /media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full.pickle


/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ten

(28800, 11)
 traces_inferences [n_frames, #_non_unique_featres]:  (28800, 11)


In [5]:
fname_out = '/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full_traces_inferences.npy'
traces_inferences = np.load(fname_out, allow_pickle=True)
print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)



 traces_inferences [n_frames, #_non_unique_featres]:  (28800, 11)


In [6]:
# #########################################################
# #########################################################
# ############## CONNECTED COMPONENTS SOLUTION ############
# #########################################################
# #########################################################

#for fname in fnames:
def connected_components_function(fname_in,
                                 ctr_comments,
                                 network_history,
                                 min_network,
                                 n_networks,
                                 start,
                                 end):
    
    fname_in = fname_in[:-7]+"_traces_inferences.npy"
    traces_inferences = np.load(fname_in, allow_pickle=True)
    print (" traces_inferences [n_frames, #_non_unique_featres]: ", traces_inferences.shape)
    print (fname_in)
    
    if end is None:
        end = traces_inferences.shape[0]
        
    # SELECT OUTPUT FILE
    fname_out = fname_in[:-4]+'_'+str(start)+'_'+str(end)+'.npz'

    #[start:end,feature]
    labels = []

    # ##########################################################
    # ################### PARAMETER LISTS ######################
    # ##########################################################
    #min_network = 5  # min length of segs to connect
    #n_networks = 4 # essentially Number of animals that are matched to a graph
    #network_history = 30

    max_dist_between_features = 125 # connected componennts max distances allowed between sequential features
    min_dist_chunks = 200 # min distance between matching history
    
    # THIS IS TRICKY: if block labeled, tough to jump 
    max_network_jump = 400 # maximum amount an animal id (centre) can move in a single frame 
    
    
    max_dist_merge_networks_cc = 75 # post-cc step to fix oversplits; searches cnetres of two networks
                                     # with non-overlapping features and merges them

    # otehr params
    min_n_matches_history_override = 4 # this parameter indicates if x feature-wise
                                       # very close matches in previous history
                                       # will override the centres-based history tracker

    # ##########################################################
    # ################## FRAME LOOP START#######################
    # ##########################################################
    comments = False

    labels_array = []
    features_array = []
    ctr=0
    historical_centres = np.zeros((network_history,n_networks,2))
    # loop over frames
    #ctr_comments =[]
    #for k in tqdm(range(start, end)):
    for k in tqdm(range(start, end)):

        if ctr in ctr_comments:
            comments=True
        else:
            comments=False

        if False:
            if k%1000==0:
                print ("FRAME: ", k)

        # make label array
        labels_array.append([])
        features_array.append([])

        # LOAD DATA FROM INFERENCE STEP
        flat_list = []
        list_idx = []
        for n, sublist in enumerate(traces_inferences[k]):
            list_idx.extend(np.zeros(len(sublist),'int32')+n)
            for item in sublist:
                flat_list.append(item)

        #print (flat_list)
        locs = np.vstack(flat_list).copy()
        list_idx = np.array(list_idx)


        # ####################################################################################
        # ####################################################################################
        # ########################### CONNECTED COMPONENTS ###################################
        # ####################################################################################
        # ####################################################################################
        # loop over features and build graphs:
        # find nearest next feature to the selected feature and connect them:
        cc = []
        cc2 = np.zeros((list_idx.shape[0],list_idx.shape[0]),'int32')
        # loop over each group of features in the dataset 
        for feature in range(np.unique(list_idx).shape[0]-1):
            idx = np.where(list_idx==feature)[0]
            locs_feat_current = locs[idx]

            # grab all features in the next step of hierarchy and mathc to previous hierarchy features;
            for qq in range(feature+1,feature+2):
                idx_next = np.where(list_idx==qq)[0]
                locs_next = locs[idx_next]

                if locs_next.shape[0]==0:
                    continue

                # Compute shortest distance between the current features and the next step features
                dists = []
                argmins = []
                for p in range(locs_feat_current.shape[0]):
                    loc_ = locs_feat_current[p]

                    vect = locs_next-loc_
                    min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                    argmin = np.argmin(scipy.spatial.distance.cdist(vect*0, vect))
                    dists.append(min_dist)
                    argmins.append(argmin)

                # ranked by distance
                dists = np.array(dists)
                argmins = np.array(argmins)
                idx_sort = np.argsort(dists)

                for p in range(idx_sort.shape[0]):
                    # load sorted data
                    min_dist = dists[idx_sort[p]]
                    argmin = argmins[idx_sort[p]]

                    if min_dist<max_dist_between_features:
                        # check to ensure the same type of feature doesn't already exist in the dataset:
                        # actually need to check if any of the other parts are connected, not just the previous most recent part?! 
                        # idx here is all the locations of current hierarchy features
                        if np.any(cc2[idx,idx_next[argmin]]):
                            if comments: 
                                print("feature already connected to other parts; skipping")
                            continue
                        #cc.append([idx[p], idx_next[argmin]])
                        cc2[idx[idx_sort[p]], idx_next[argmin]]=1


        graph=np.array(cc2)
        graph = csr_matrix(graph)


        n_components, labels_all = connected_components(csgraph=graph, directed=False, return_labels=True)
        #print (n_components)

        # find all unique network ids; labels_unique[0] - ids;  labels_unique[1] is counts
        labels_unique = np.unique(labels_all, return_counts=True)

        # keep only networks over min_network size;
        idx_nets = np.where(labels_unique[1]>=min_network)[0]
        labels_kept = labels_unique[0][idx_nets]

        # order the networks by size and keep only to n_networks
        idx_ordered = np.argsort(labels_unique[1][idx_nets])[::-1]
        labels_kept = labels_kept[idx_ordered][:n_networks]
        if comments:
            print ("idx_networks kept:", idx_nets)
            print ("labels kept: ", labels_kept)
            print (" size of networks kept: ", labels_unique[1][idx_nets][idx_ordered][:n_networks])

            for p in np.unique(labels_kept):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]
                print ("network :", p, locs_network)
                print ('size of network ', locs_network.shape)
                print (" feature types; ", list_idx[idx])
            print ("")

        # final cc check for oversplit clusters:
        # note: This should be done in the CC matrix step, but can't seem to get it to work in full
        labels_kept_temp = labels_kept.copy()
        for p in np.unique(labels_kept):
            idx1 = np.where(labels_all==p)[0]
            features1 = list_idx[idx1]
            locs_network1 = locs[idx1]

            if idx1.shape[0]==0:
                continue

            for pp in np.unique(labels_kept):
                if pp==p:
                    continue
                idx2 = np.where(labels_all==pp)[0]
                if idx2.shape[0]==0:
                    continue

                features2 = list_idx[idx2]
                locs_network2 = locs[idx2]

                # check if nonoveralpping features
                if np.any(np.isin(features1,features2))==False:

                    # chekc if very close in distance
    #                 vect = labels_array[ctr-1][z]-locs_feature
    #                 min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                    dists = scipy.spatial.distance.cdist(locs_network1, locs_network2)
                    if comments:
                        print ("dists between nonvoeralpping networks: ", dists)

                    # merge bits
                    if np.min(dists)<max_dist_merge_networks_cc:

                        # change labels_all and labels_kept
                        labels_all[idx2] = p
                        idx_del = np.where(labels_kept_temp==pp)[0]
                        labels_kept_temp = np.delete(labels_kept_temp, idx_del)

                        if comments:
                            print ("merged networks: ", p, " and ", pp, " at idx: ", idx_del)


        labels_kept = labels_kept_temp
        #print ("LABELS post cc: ", labels)

        # ####################################################################################
        # ####################################################################################
        # ########################### FRAME LOOP ANALYSIS ####################################
        # ####################################################################################
        # ####################################################################################
        # save previous centres to match them and overwrite below
        # loop over all network components
        if ctr>0:
            # make large list to hold ftuure labels
    #        features_list = [
            for p in range(n_networks):
                labels_array[ctr].append([])
                features_array[ctr].append([])

            # keep track of the matches with previous chunks
            previous_centres = historical_centres.copy() #[0,p]=np.mean(locs_network,0))

            if comments:
                print ("previous centres: ", previous_centres.shape)
            #break

            idx_matches = []
            idx_matches_p = []


            # ####################################################################################
            # ########################### COMPUTE DISTANCES TO PREVIOUS FRAME ####################
            # ####################################################################################

            # search for current network chunk for nearest previous chunk
            # count the number of matches between current network centre and history:
            n_matches_history = np.zeros((labels_kept.shape[0], previous_centres.shape[1]),'int32')
            total_dist = np.zeros((labels_kept.shape[0],previous_centres.shape[1]),'float32') 

            # new method checks averages
            # and also whether min_network closest matches to the previous immediate vals;
            # If option 2 is very close it overrides all other matches. 
            for ctr_p, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]

                # compute cntre:
                centre_network = np.median(locs_network,0)
                if comments:
                    print ("centre network", p, centre_network)
                    print (" doing feature-wise heuristic search at prev step: ")
                    #print ("      Previous time step data", labels_array[ctr-1])
                    #print ("      current network: ", locs_network)
                # also quick check if the previous steps had values very close to current 
                # loop over current network values and find shortest 5 distances:
                min_dist_count = np.zeros(n_networks, 'int32')
                for zz in range(locs_network.shape[0]):
                    locs_feature = locs_network[zz]
                    if comments:
                        print ("search for close feature to ", locs_feature)
                        #if comments:

                    #min_dist_count = 0
                    for z in range(len(labels_array[ctr-1])):
                        temp_temp = labels_array[ctr-1][z]
                        if len(labels_array[ctr-1][z])==0:
                            continue
                        vect = labels_array[ctr-1][z]-locs_feature
                        min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                        #print ("min dist: ", min_dist)
                        if min_dist < min_n_matches_history_override:
                            min_dist_count[z]+=1

                # if sufficient matches no need to compute and check centres
                max_nearest = np.max(min_dist_count)
                if max_nearest > 5:
                    argmax_overwrite = np.argmax(min_dist_count)
                    #n_matches_history[:,argmax_overwrite]-=100  # ensure the previous neetwork can only be matched to the current one
                                                      # by starting/biasing the match count for the rest of the matches;
                                                      # can also do this other ways
                    n_matches_history[ctr_p,argmax_overwrite]+=1E5
                    total_dist[ctr_p,argmax_overwrite]=0 #previous_c

                    if comments:
                        print ("nearly similar network at previous time step; ")
                        print (" Min dist array: ", min_dist_count)
                        print ("         matching to prev network using argmax: ", argmax_overwrite)
                        print ("         current locs ", locs_network)
                        print ("         previous matched network: ", labels_array[ctr-1][argmax_overwrite])

                else:
                    # loop over each previous network
                    for z in range(previous_centres.shape[1]):

                        # loop over each previous history point: find how many historical matches it has
                        for h in range(previous_centres[:,z].shape[0]):

                            # May wish to implement an L1 not L2 distance 
                            # OR an individual feature distance rather than mean
                            # OR find median xy point and use that as centre;
                            #if True:
                            vect = previous_centres[h,z]-centre_network
                            min_dist = np.min(np.linalg.norm(vect))
                            #print ("VECT: ", vect, " Min dist: ", min_dist)
                            #min_dist = np.min(scipy.spatial.distance.cdist(vect*0, vect))
                            if min_dist<min_dist_chunks:
                                n_matches_history[ctr_p][z]+=1
                                total_dist[ctr_p][z]+=min_dist #previous_centres[:,z].shape[0]-h)/previous_centres[:,z].shape[0])
                                #total_dist[z]+=min_dist*((previous_centres[:,z].shape[0]-h)/previous_centres[:,z].shape[0])

                                if comments:
                                    print (z,h,"adding dist: ", min_dist)

            # check best matches from history
            for ctr_p, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]

                if comments:
                    print ("")
                    print ("animal:", ctr_p, "n_mathces_history:", n_matches_history[ctr_p])
                    print ("locs: ", locs_network)

                # Loop to ensure no better match found for a previous animal ID        
                while True:
                    argmax = np.argmax(n_matches_history[ctr_p])
                    if comments:
                        print ("curretn best match for current locs network: ", locs_network)
                        print ("   is prev network: ", labels_array[ctr-1][argmax])

                    if n_matches_history[ctr_p,argmax]==0:
                        break

                    # ensure no boetter match was found for the current val
                    n_matches_all_column = n_matches_history[:,argmax]
                    if np.max(n_matches_all_column)>n_matches_history[ctr_p,argmax]:

                        argmax_col = np.argmax(n_matches_all_column)

                        # check also distances for better fit:
                        current_dist = total_dist[ctr_p][argmax]/n_matches_history[ctr_p][argmax]
                        alternative_match_dist = total_dist[argmax_col][argmax]/n_matches_history[argmax_col][argmax]

                        if comments:
                            print (" current dist: ", current_dist)
                            print (" alternative dist: ", alternative_match_dist)
                        
                        #if False:
                        if alternative_match_dist< current_dist:

                            n_matches_history[ctr_p, argmax]=0
                            if comments: 
                                print (" FOUND BETTER MACH WITH ANOTHER ANIMAL, setting argmax to zoer")
                        else:
                            break
                        #else:
                        #    break
                    else:
                        break


                # check case when more than 1 animal within merging distance and pick closest
                max_count = n_matches_history[ctr_p][argmax]
                if comments:
                    print (" argmax in match history: ", argmax)
                    print ('               max count: ', max_count)

                # check if more than one match for new network
                # may wish to take all matches > 15 or max

                idx11 = np.where(n_matches_history[ctr_p]==max_count)[0]
                if (idx11.shape[0]>1) and max_count>0:

                    # find what the averate distance between each matched history and the new frame are and 
                    # take the shortest average distance
                    # check all the matches that had same amount as idx11
                    match_means = total_dist[ctr_p][idx11]/n_matches_history[ctr_p][idx11]
                    argmin_match = np.argmin(match_means)
                    argmax = idx11[argmin_match]
                    min_dist_found = match_means[argmin_match]

                    if comments:
                        print ("DUPLICATE MATCHES: so looking at distances: ", total_dist[ctr_p][idx11])
                        print ("Original min distance found: ", min_dist_found)
                        print ("Average distancesL", match_means, " of argmin: ", argmin_match)
                        print ("total dists: ", total_dist[ctr_p])

                    # check if matched network doesn't actually have even closer match:
                    # so search in the column of the original best match to see if it has a better match
                    idx13 = np.where(n_matches_history[:,argmax]==max_count)[0]
                    match_means_network_match = total_dist[:,argmax][idx13]/n_matches_history[:,argmax][idx13]
                    argmin_match_near = np.argmin(match_means_network_match)
                    if match_means_network_match[argmin_match_near]<min_dist_found:

                        # set the distance for the previous best match to very large value
                        total_dist[ctr_p][argmax] = 1E10

                        # recompute the argmax    
                        match_means = total_dist[ctr_p][idx11]/n_matches_history[ctr_p][idx11]
                        argmin_match = np.argmin(match_means)
                        argmax = idx11[argmin_match]
                        min_dist_found = match_means[argmin_match]

                        if comments:
                            print ("BETTER MATCH WAS FOUND", argmin_match)
                            print ("new min distance found: ", min_dist_found)


                        #else:
                        #    break
                    else:
                        if comments:
                            print ("NO BETTER MATCH FOUND")

                # check also to see if networked matched on centre-based distance doesn't actually ahve a better match via
                #   feature-based matching with another dataset;
                n_matches_all_column = n_matches_history[:,argmax]
                if np.max(n_matches_all_column)>n_matches_history[ctr_p,argmax]:
                    if comments:
                        print ("  Matches in columN: ", n_matches_all_column)
                        print ("     ARE GREATER THAN BEST Centre based match: ", n_matches_history)
                        print ("  ***************************************")

                # ############################################################ 
                # ######################## SAVE MATCH DATA ################### 
                # ############################################################ 
                if n_matches_history[ctr_p][argmax]>0:
                    # replace locations of previous match 
                    idx_matches.append(argmax)
                    idx_matches_p.append(p)
                    if comments:
                        print ("Matched to previous centre: ", argmax)
                        print ("final argmax: ", argmax)
                        print ("locs current network:", locs_network)
                        print ("to be inserted near previous network: ", labels_array[ctr-1][argmax])

                # add it as a new tracklet
                else:
                    idx_matches.append(None)
                    idx_matches_p.append(p)

                # zero out the better match in this column
                total_dist[:,argmax]=1E10
                n_matches_history[:,argmax] = 0

            # ############################################################
            # ############################################################
            # ############### INSERT THE MATCHED DATA ####################
            # ############################################################
            # ############################################################
            if comments:
                print("")
                print("FINAL NETWORK INSERTION STEP")

            # loop over the matched ids
            for p in range(len(idx_matches)):
                idx = np.where(labels_all==idx_matches_p[p])[0]
                locs_network = locs[idx]
                feature_types = list_idx[idx]
                if comments:
                    print ("idx_matches[p]:", idx_matches[p])

                if idx_matches[p] is not None:

                    # do a final check to make sure neighbouring timestep networks aren't super-far apart
                    centre_prev_network = np.median(labels_array[ctr-1][idx_matches[p]],0)
                    centre_current_network = np.median(locs_network,0)
                    vect = centre_prev_network-centre_current_network
                    dist_centre = np.linalg.norm(vect)
                    if comments:
                        print ("   FINAL CHECK DISTANCE BETWEEN PREVIOUS AND CURRENT CENTRE",
                              "  vect" , vect,
                              "  dist: ", dist_centre)
                    if dist_centre>=max_network_jump:
                        if comments:
                            print (" SKIPPED NETWORK DUE TO LARGE JUMP!")
                    else:
                        labels_array[ctr][idx_matches[p]]=locs_network
                        features_array[ctr][idx_matches[p]]=feature_types

                        if comments:
                            print ("")
                            print ("inserted: ", locs_network)
                            print (" at location: ", idx_matches[p])
                            print ("ctr-1 values at this location: ",idx_matches[p],
                                   " has values: ",
                                   labels_array[ctr-1][idx_matches[p]])
                else:
                    if comments:
                        print ("network doesn't have match: ", locs_network)

            # ############################################################
            # ############################################################
            # ############### REVIEW UNMATCHED DATA ####################
            # ############################################################
            # ############################################################           
            ctr_inner=0
            for p in range(len(idx_matches)):
                if idx_matches[p] is None:
                    idx = np.where(labels_all==idx_matches_p[p])[0]
                    feature_types = list_idx[idx]
                    locs_network = locs[idx]
                    if comments:
                        print ("Inserting missed network to end:", locs_network)
                    # search for an empty list to populate
                    while True:
                        # find the first empty location to insert data
                        if len(labels_array[ctr][ctr_inner])==0:

                            # do a final check to make sure neighbouring timestep networks aren't super-far apart
                            if len(labels_array[ctr-1][ctr_inner])==0:
                                centre_prev_network=np.array((1E10,1E10))
                            else:
                                centre_prev_network = np.median(labels_array[ctr-1][ctr_inner],0)
                            
                            vect = centre_prev_network-centre_current_network
                            dist_centre = np.linalg.norm(vect)
                            if comments:
                                print ("   FINAL CHECK DISTANCE BETWEEN PREVIOUS AND CURRENT CENTRE",
                                      "  vect" , vect,
                                      "  dist: ", dist_centre)
                            if dist_centre>=max_network_jump:
                                if comments:
                                    print (" SKIPPED NETWORK DUE TO LARGE JUMP!")

                                # exit and skip this segment altogether
                                break

                            else:
                                labels_array[ctr][ctr_inner]=locs_network
                                features_array[ctr][ctr_inner]=feature_types
                                ctr_inner+=1
                                break
                        ctr_inner+=1

            # add centres to 

            historical_centres[1:]=historical_centres[:-1] #[0,p]=np.mean(locs_network,0))
            for pp in range(len(labels_array[ctr])):
                if len(labels_array[ctr][pp])==0:
                    continue
                locs_new = labels_array[ctr][pp]
                centres_new = np.median(locs_new,0)
                if comments:
                    print ("centres new", pp, centres_new)
                #historical_centres[0,pp]=centres_new
                historical_centres[0,pp]=centres_new


            if comments:
                print (k, " FINAL labels array: ", labels_array[ctr])

        # for first time point
        else:
            for k in range(n_networks):
                labels_array[ctr].append([])
                features_array[ctr].append([])

            for n, p in enumerate(np.unique(labels_kept)):
                idx = np.where(labels_all==p)[0]
                locs_network = locs[idx]
                feature_types = list_idx[idx]

                #print ("locs network", locs_network)
                labels_array[ctr][n]=locs_network
                features_array[ctr][n]=feature_types

                # fill the entire history with starting locations:
                historical_centres[:,n]=np.median(locs_network,0)

            if comments:
                print ("FIRST saved labels array: ", labels_array)
    #             break
    #         break
    #     break

        ctr+=1   
        #print ("")
        #print ("")

    # convert the output to a rectangular matrix with correct locations for data;
    final_features = np.zeros((n_networks*14, len(labels_array),2), 'float32')
    print ("Final eatures; ", final_features.shape)
    for k in range(len(labels_array)):
        if k%1000==0:
            print (k)
        for p in range(len(labels_array[k])):
            #for l in range(len(labels_array[k][p])):
                #print (data[k][p][l])
            label_locs = features_array[k][p]
            if len(label_locs)>0:
                #print (k, p, label_locs)
                final_features[label_locs+p*14,k]=labels_array[k][p]

    tracesx = final_features[:,:,0]
    tracesy = final_features[:,:,1]
    probs = np.ones(tracesy.shape, 'float32')
    print ("SAVING:to file: ", fname_out)
    np.savez(fname_out,
            tracesx=tracesx,
            tracesy=tracesy,
            probs =probs
            )    
    
    return fname_out
    


In [51]:
# # make inference traces arrays from Anqi's data
# fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/markers/markers'

# inferences = []
# for k in range(14):
#     fname_temp = fname + str(k)+'.npy'
#     temp = np.load(fname_temp)
#     inferences.append(temp)
# inferences=np.array(inferences).transpose(1,0,2,3)
# print (inferences.shape)

# np.save('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/markers/markers_inferences_traces.npy', inferences)


In [7]:
#fname = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full.pickle'
fname = '/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full.pickle'
# fname_in = fname_in[:-7]+"_traces_inferences.npy"

from tqdm import tqdm

ctr_comments = []
network_history = 5
min_network= 4
n_networks = 7
start = 0
end = 1000
fname_out = connected_components_function(fname,
                                          ctr_comments,
                                          network_history,
                                          min_network,
                                          n_networks,
                                          start,
                                          end
                                         )
# PARALLEL VERSION OF CC OVER FILES (NOT FRAMES)
#fnames = ['/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_1000frames.pickle']#

#if True:
#    import parmap
#    parmap.map(connected_components_function, fnames,
#              pm_processes=4)


  1%|          | 6/1000 [00:00<00:16, 59.26it/s]

 traces_inferences [n_frames, #_non_unique_featres]:  (28800, 11)
/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full_traces_inferences.npy


  1%|          | 12/1000 [00:00<00:17, 57.75it/s]/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 1000/1000 [00:17<00:00, 55.76it/s]


Final eatures;  (98, 1000, 2)
0
SAVING:to file:  /media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full_traces_inferences_0_1000.npz


In [35]:
################################################################################
################################################################################
######################### VISUALIZE 2D LOCATIONS ###############################
################################################################################
################################################################################
clrs_new = ['blue','red','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']
frame_id = 0
offset = 40
# load video data
video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(video_name)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start+offset)

# load traces
data = np.load(fname_out)
tracesx = data['tracesx'].T
tracesy = data['tracesy'].T

labels_array = np.array((tracesx, tracesy))
labels_array = labels_array.transpose(1,2,0)
print ("Labels array: ", labels_array.shape)

#scale_percent = 400 # percent of original size
width = 320*4
height = 256*4
dim = (width, height)

fig=plt.figure()
plt.tight_layout(pad=1.08, h_pad=None, w_pad=None, rect=None)

for ctr_plot, t in enumerate(np.arange(0+offset,end-start+offset,1)):
    ax=plt.subplot(2,5,ctr_plot+1)
    #ax=plt.subplot(1,2,t+1)
    
    ret, frame = original_vid.read()
    
    # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA) 

    plt.imshow(resized,alpha=.7)

    for a in range(0,labels_array.shape[1],14):
        print (a)
        tempx = labels_array[t, a:a + 14][:,0]
        tempy = labels_array[t, a:a+14][:,1]
        
        print (tempx)
        plt.scatter(
            tempx, 
            tempy, 
            c=clrs_new[a//14],alpha=.8)
    
    plt.title(str(t+frame_id)+" " +str(round((t+frame_id)/25.,1)),fontsize=8)
#     plt.ylim(1024,0)
#     plt.xlim(0,1280)
    plt.xticks([])
    plt.yticks([])
plt.show()


Labels array:  (100, 70, 2)
0
[378.959 355.665 359.257 326.89  332.002 317.228 293.326 268.85  257.51
   0.    245.954 237.661 226.822 226.52 ]
14
[1027.058 1009.102 1019.859 1010.843 1044.049 1029.055 1025.898 1025.063
 1019.651 1009.94  1007.614 1003.3   1000.934  998.27 ]
28
[   0.       0.    1164.489 1180.25  1156.742 1161.645 1147.505 1134.365
 1121.112 1112.532 1099.776 1093.513    0.       0.   ]
42
[   0.       0.       0.    1142.271 1170.475 1147.864 1140.955 1138.411
 1125.099 1100.705    0.       0.       0.       0.   ]
56
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0
[379.025 356.079 362.445 327.023 332.006 317.167 293.349 268.727 258.498
   0.      0.      0.      0.      0.   ]
14
[ 979.877  988.323 1020.102 1001.03  1035.045 1018.207 1019.509 1021.541
 1019.806 1005.538 1001.532  994.607  986.272  982.961]
28
[   0.       0.    1163.764 1180.338 1156.94  1162.136 1146.138 1132.008
 1116.756 1107.67  1099.636 1093.288 1092.592    0.   ]
42
[   0.       0.       0.    1

ValueError: num must be 1 <= num <= 10, not 11

In [8]:
frame_id = 583
#print (labels_array[200])
#clrs_new = ['blue','red','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']


fig=plt.figure()
frame_ids = [730]
for frame_id in frame_ids:
    for k in range(len(labels_array[frame_id])):
        print (frame_id, k)
        print (labels_array[frame_id][k])
        if len(labels_array[frame_id][k])>0:
            plt.scatter(np.array(labels_array[frame_id][k])[:,0], 
                    np.array(labels_array[frame_id][k])[:,1],
                    c=clrs_new[k])
        #print (np.array(labels_array[frame_id+t][k]).shape)
        if np.array(labels_array[frame_id][k]).shape[0]>0:
            plt.scatter(np.array(labels_array[frame_id][k])[:,0].mean(0),
              np.array(labels_array[frame_id][k])[:,1].mean(0),
              s=100, c=clrs_new[k])

frame_ids = [731]
for frame_id in frame_ids:
    for k in range(len(labels_array[frame_id])):

        if np.array(labels_array[frame_id][k]).shape[0]>0:
            plt.scatter(np.array(labels_array[frame_id][k])[:,0].mean(0),
              np.array(labels_array[frame_id][k])[:,1].mean(0),
              s=100, c=clrs_new[k])


    
#plt.title(str(t+frame_id))
plt.ylim(1024,0)
plt.xlim(0,1280)
plt.show()

730 0
[0. 0.]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [7]:
traces_inferences = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npy',allow_pickle=True)

print (traces_inferences.shape)


(89988, 14)


In [9]:
#########################################################################
######################### MAKE VIDEOS ###################################
#########################################################################

# Add fixed labels to video:
reassembled = np.load('/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727_compressedDLC_resnet50_cohort2Oct21shuffle1_200000_full_traces_inferences_0_1000.npz')
tracesx_re = reassembled['tracesx'].T
tracesy_re = reassembled['tracesy'].T
print (tracesx_re.shape)

# OPTIONAL MAKE VIDEO TO REVIEW ASSEMBLED VS. INFERENCE LABELS (PRE-FIX)
# colors have weird inversion; red is blue and cyan is yellow
#colors_4 = ['blue','red','cyan','green','pink','orange']

#          pup1     pup2    female  male
colors_4= ['orange','green', 'blue', 'red', 'cyan','white','yellow']

video_name = '/media/cat/4TBSSD/dan/cohort2/2020_07_19_16_32_55_857727.avi'
original_vid = cv2.VideoCapture(video_name)

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_corrected.mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)

#dot_size = 10//scale
# setup cutoff 
pcutoff = 0.01

# go through first videos
from tqdm import trange

original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

font = cv2.FONT_HERSHEY_PLAIN

for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    #cv2.putText(frame,   text,location,  font,font size, font color,    font weight, line)
    #cv2.putText(img, text, pos, font_face, scale, color, 1, cv2.LINE_AA)
    
#     for k in range(traces_inferences.shape[1]):
#         for p in range(len(traces_inferences[n,k])):
#             y = int(traces_inferences[n][k][p][0])//scale + 2
#             x = int(traces_inferences[n][k][p][1])//scale + 2
         
#             frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
#                 matplotlib.colors.to_rgb('white'))*255.).astype('uint8')
    
    
    for k in range(14*n_networks):
        y = tracesx_re[n,k]
        x = tracesy_re[n,k]
        
        if np.isnan(x) or np.isnan(y):
            continue
        else:
            x=int(x)//scale
            y=int(y)//scale
            
            frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(colors_4[k//14]))*255.).astype('uint8')
            #print (colors_4[k])
            #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()


  1%|          | 9/1000 [00:00<00:11, 82.75it/s]

(1000, 98)


100%|██████████| 1000/1000 [00:12<00:00, 78.56it/s]


In [32]:
data1 = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/markers/markers_traces_inferences.npz')
tracesx = data1['tracesx']
print (tracesx.shape)

data2 = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_1000frames_traces_inferences.npz')
tracesx = data2['tracesx']
print (tracesx.shape)


(56, 337)
(56, 1000)
